### PointNet to Classify Proteins

#### Dataset study

In [1]:
# Getting the data
import os
from pandas import DataFrame
import pandas as pd

from datasetStudy import *

from pathlib import Path

debug_pointnet = False

# Multiple function here depends on thi variable
#root = '/mnt/dataset/shrec-2025-protein-classification/v2-20250331' 
base_path = Path('/mnt/')
#base_path = Path('/mnt')
root = base_path / 'dataset/shrec-2025-protein-classification/v2-20250331' 

#train_data = os.listdir(os.path.join(root, 'train'))
#train_data_cls = pd.read_csv('datasets/train_set-all.csv', sep=',', index_col=0)

train_data = os.listdir(root / 'train')
train_data_cls = pd.read_csv('./datasets/train_set-all.csv', sep=',', index_col=0)

print(train_data[:5])
print()
print(train_data_cls.info())
train_data_cls

['8ugd_8:R:3U_model1.vtk', '8h0v_18:R:c_model1.vtk', '3j3q_1:DX:4F_model1.vtk', '4u4u_23:XC:d1_model1.vtk', '6rny_4:H:H_model1.vtk']

<class 'pandas.core.frame.DataFrame'>
Index: 9244 entries, 0 to 9243
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   protein_id        9244 non-null   object
 1   class_id          9244 non-null   int64 
 2   number_of_points  9244 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 288.9+ KB
None


,protein_id,class_id,number_of_points
0,8ugd_8:R:3U_model1,96,5916
1,8h0v_18:R:c_model1,86,10078
2,3j3q_1:DX:4F_model1,8,18432
3,4u4u_23:XC:d1_model1,83,8242
4,6rny_4:H:H_model1,34,9204
...,...,...,...
9239,3j3y_1:HL:6R_model1,8,18342
9240,4u4y_15:O:C3_model1,91,12976
9241,7w31_26:AA:c_model1,18,16454
9242,3j4k_1:E:E_model1,90,23188


In [2]:
possible_disconnected_mesh(train_data_cls, 96, False)

disconnected_dict = {}
for idx in range(97):
    disconnected_dict[idx] = possible_disconnected_mesh(train_data_cls, idx)

#print(disconnected_dict)

_, damaged = possible_disconnected_mesh(train_data_cls, 8, True); damaged

,protein_id,class_id,number_of_points


In [3]:
dist = cls_distribution(train_data_cls)
inspect_distribution(dist, l_lim=0, u_lim=5)

Classes which have between 0 and 5 element: 16/97


In [4]:
dist_all = cls_distribution(train_data_cls)

raw_train_dataframe_f1 = number_of_point_filter(train_data_cls, 1000)
dist_f1 = cls_distribution(raw_train_dataframe_f1)

raw_train_dataframe_f2 = number_of_class_filter(raw_train_dataframe_f1, 10)
dist_f2 = cls_distribution(raw_train_dataframe_f2)

raw_train_dataframe_f3 = number_of_class_filter(raw_train_dataframe_f1, 10, 100)
dist_f3 = cls_distribution(raw_train_dataframe_f3)

raw_train_dataframe_f4 = number_of_class_filter(raw_train_dataframe_f1, 10, 50)
dist_f4 = cls_distribution(raw_train_dataframe_f4)

raw_train_dataframe_f5 = number_of_class_filter(raw_train_dataframe_f1, 100)
dist_f5 = cls_distribution(raw_train_dataframe_f5)

distributions = [dist_all, dist_f1, dist_f2, dist_f3, dist_f4, dist_f5]
for idx in range(len(dist_all)):
    output = f'cls {idx}: '
    for dist in distributions:
        output += f'{print_dist(dist, idx)}\t'
    print(output)

print(f'{len(train_data_cls) = }')
print(f'{len(raw_train_dataframe_f1) = }')
print(f'{len(raw_train_dataframe_f2) = }')
print(f'{len(raw_train_dataframe_f3) = }')
print(f'{len(raw_train_dataframe_f4) = }')
print(f'{len(raw_train_dataframe_f5) = }')

cls 0: 101	101	101	0	0	101	
cls 1: 3	3	0	0	0	0	
cls 2: 17	17	17	17	17	0	
cls 3: 21	21	21	21	21	0	
cls 4: 15	15	15	15	15	0	
cls 5: 127	127	127	0	0	127	
cls 6: 25	25	25	25	25	0	
cls 7: 75	75	75	75	0	0	
cls 8: 2054	2054	2054	0	0	2054	
cls 9: 66	66	66	66	0	0	
cls 10: 14	14	14	14	14	0	
cls 11: 43	43	43	43	43	0	
cls 12: 10	10	10	10	10	0	
cls 13: 22	22	22	22	22	0	
cls 14: 489	489	489	0	0	489	
cls 15: 89	89	89	89	0	0	
cls 16: 154	154	154	0	0	154	
cls 17: 116	116	116	0	0	116	
cls 18: 76	76	76	76	0	0	
cls 19: 42	42	42	42	42	0	
cls 20: 7	7	0	0	0	0	
cls 21: 74	74	74	74	0	0	
cls 22: 58	58	58	58	0	0	
cls 23: 10	10	10	10	10	0	
cls 24: 10	10	10	10	10	0	
cls 25: 143	143	143	0	0	143	
cls 26: 2	2	0	0	0	0	
cls 27: 3	3	0	0	0	0	
cls 28: 18	18	18	18	18	0	
cls 29: 14	14	14	14	14	0	
cls 30: 3	3	0	0	0	0	
cls 31: 33	33	33	33	33	0	
cls 32: 93	93	93	93	0	0	
cls 33: 126	126	126	0	0	126	
cls 34: 73	73	73	73	0	0	
cls 35: 23	23	23	23	23	0	
cls 36: 8	8	0	0	0	0	
cls 37: 119	119	119	0	0	119	
cls 38: 34	34	34	34	34	0	
cls

In [5]:
cls_selected = [8, 90, 56, 14, 61, 70, 86, 5, 16, 25, 54, 62, 74, 83, 91, 92, 17, 45, 87, 71]

df_3 = create_dataframe(raw_train_dataframe_f1, cls_selected[:2], 500)
df_4 = create_dataframe(raw_train_dataframe_f1, cls_selected[:10], 100)
df_5 = create_dataframe(raw_train_dataframe_f1, cls_selected[:20], 50)
df_6 = create_dataframe(raw_train_dataframe_f1, cls_selected[:20], 100)

'''
df_3.to_csv('datasets/train_set-2_cls-1000_images.csv')
df_4.to_csv('datasets/train_set-10_cls-1000_images.csv')
df_5.to_csv('datasets/train_set-20_cls-1000_images.csv')
df_6.to_csv('datasets/train_set-20_cls-2000_images.csv')
'''

"\ndf_3.to_csv('datasets/train_set-2_cls-1000_images.csv')\ndf_4.to_csv('datasets/train_set-10_cls-1000_images.csv')\ndf_5.to_csv('datasets/train_set-20_cls-1000_images.csv')\ndf_6.to_csv('datasets/train_set-20_cls-2000_images.csv')\n"

#### Augmentations

In [6]:
from format import Text
import torch
from symmetria.transformations import *
from symmetria.shapes import BenchmarkShape

In [7]:
points_rectangle = np.array([[0, 2, 0, 0, 2, 2, 0, 2],
                             [0, 0, 1, 0, 1, 0, 1, 1],
                             [0, 0, 0, 1, 0, 1, 1, 1]]).astype(np.float32)

points_rectangle_t = torch.from_numpy(points_rectangle)

rot = random_rotation_matrix(rotate_x=False, rotate_z=False)

shapebench = BenchmarkShape(points_rectangle_t)
#shapebench.apply_rotation(torch.from_numpy(rot))
#shapebench.apply_traslation(1,1,0)
#shapebench.apply_uniform_noise(100, 1)
#shapebench.apply_gaussian_noise(100, 1)

rect = pv.PolyData(shapebench.points.numpy().T)
#rect.plot()

In [99]:
from random import random

class RotateAroundZero():
    def __init__(self, p=0.5, rot=None):
        self.p = p
        self.rot = rot

    def __call__(self, points):
        if random() < self.p:
            if self.rot == None:
                rot = torch.from_numpy(random_rotation_matrix())
            else:
                rot = self.rot
        
            rot = rot.to(torch.device(points.device))

            ones = torch.ones((1,points.shape[0]), device=points.device)
            coords = torch.concatenate((torch.transpose(points, 0, 1), ones))
            
            coords = rot@coords
            
            points = torch.transpose(coords[:3,:], 0, 1)

        return points
    
    def __repr__(self):
        return f'RotateAroundZero(p={repr(self.p)}, rot={repr(self.rot)})'
    
class Translate():
    def __init__(self, p=0.5, shift=None, scale=1):
        self.p = p
        self.shift = shift
        self.scale = scale

    def __call__(self, points):        
        if random() < self.p:
            if self.shift == None:
                shift = (torch.rand(1, device=points.device) * self.scale,
                         torch.rand(1, device=points.device) * self.scale,
                         torch.rand(1, device=points.device) * self.scale)
            else:
                shift = self.shift
            
            for i in range(len(shift)):
                points[:,i] += shift[i]
                   
        return points
    
    def __repr__(self):
        return f'Translate(p={repr(self.p)}, shift={repr(self.shift)}, scale={repr(self.scale)})'
    
class UniformNoise():
    def __init__(self, n, T, p=0.5):
        self.p = p
        self.n, self.T = n, T
    
    def __call__(self, points):
        if random() < self.p:
            num_points = points.shape[0]

            indices = torch.randperm(num_points, device=points.device)[:self.T]

            points[indices,:] = points[indices,:] + (2*torch.rand(self.T, 3, device=points.device)-1)/self.n
        
        return points
    
    def __repr__(self):
        return f'UniformNoise(n={repr(self.n)}, T={repr(self.T)}, p={repr(self.p)})'
    
class GaussianNoise():
    def __init__(self, n, T, p=0.5):
        self.p = p
        self.n, self.T = n, T
    
    def __call__(self, points):
        if random() < self.p:
            num_points = points.shape[0]

            indices = torch.randperm(num_points, device=points.device)[:self.T]
    
            points[indices,:] = points[indices,:] + torch.rand(self.T, 3, device=points.device)/self.n

        return points
    
    def __repr__(self):
        return f'GaussianNoise(n={repr(self.n)}, T={repr(self.T)}, p={repr(self.p)})'

In [100]:
points_rectangle = np.array([[0, 2, 0, 0, 2, 2, 0, 2],
                             [0, 0, 1, 0, 1, 0, 1, 1],
                             [0, 0, 0, 1, 0, 1, 1, 1]]).astype(np.float32).T

points_rectangle_t = torch.from_numpy(points_rectangle).to(torch.device('cuda:0'))

translate = Translate(p=1)
print(translate)
points_rectangle_t = translate(points_rectangle_t)
print(translate)
rect = pv.PolyData(points_rectangle)
#rect.plot()

rect_tfm = pv.PolyData(points_rectangle_t.cpu().numpy())
#rect_tfm.plot()

Translate(p=1, shift=None, scale=1)
Translate(p=1, shift=None, scale=1)


#### DataSet and DataLoader

In [ ]:
import numpy as np
import lzma
from torch.utils.data import Dataset, DataLoader
from symmetria.transforms.RandomSampler import RandomSampler
from symmetria.transforms.UnitSphereNormalization import UnitSphereNormalization
from tqdm import tqdm

class ProteinDataset(Dataset):
    def __init__(self, data_df, tfms, root='/mnt/dataset/shrec-2025-protein-classification/v2-20250331', extention='xz', train=True):
        super().__init__()
        self.df = data_df
        self.tfms = tfms
        self.extention = extention
        
        self.train = train
        if self.train:
            if self.extention == 'vtk':
                self.root = os.path.join(root, 'train')

            elif self.extention == 'xz':
                self.root = os.path.join(root, 'train-xz')
            
            else:
                raise UserWarning('Extention not supported')
            
        else:
            raise UserWarning('Not yet implemented for the test dataset')
        
        self.encode_label()
        self.loader()
    
    def loader(self):
        self.data = []
        
        for index in tqdm(self.df.index):
            protein, cls, nop = self.df['protein_id'].loc[index], self.df['class_id'].loc[index], self.df['number_of_points'].loc[index]

            cls_t = torch.tensor(self.encoded_cls[cls]).to(torch.device('cuda:0'))

            if self.extention == 'vtk':
                point_cloud = self.get_vtk_points(protein)

            elif self.extention == 'xz':
                point_cloud = self.get_xz_points(protein, cls, nop)

            point_cloud_t = torch.from_numpy(point_cloud).to(torch.device('cuda:0'))     

            point_cloud_t, cls_t = point_cloud_t.type(torch.float32), cls_t.type(torch.float32)
            self.data.append((point_cloud_t, cls_t))

    def get_vtk_points(self, name):
        prot_file = name + '.' + self.extention
        prot_file = os.path.join(self.root, prot_file)
        
        prot_mesh = pv.read(prot_file)

        return prot_mesh.points

    def get_xz_points(self, name, cls, nop):
        cls, nop = str(cls), str(nop)

        while len(cls) < 2:
            cls = '0' + cls
        
        while len(nop) < 6:
            nop = '0' + nop

        prot_file = cls + '-' + nop +  '-' + name.replace(':', '+') + '.' + self.extention
        prot_file = os.path.join(self.root, prot_file)        
        
        with lzma.open(prot_file, 'rt') as f:
            point_cloud = np.loadtxt(f)

        return point_cloud

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index, raw=False):
        prot, cls = self.data[index]
        
        if not raw:
            for tfm in self.tfms:
                prot = tfm(prot)

        prot = torch.transpose(prot, 0, 1)
        return (prot, cls)
    
    def encode_label(self):
        self.encoded_cls = {}
        
        prot_clss = np.sort(self.df['class_id'].unique())
        
        #One hot encoding
        '''
        for idx, cls in enumerate(prot_clss):
            self.encoded_cls[cls] = np.eye(len(prot_clss))[idx]
        '''

        for idx, cls in enumerate(prot_clss):
            self.encoded_cls[cls] = int(idx)

    def render_pointcloud(self, index):
        prot, _ = self.data[index]
        prot = torch.transpose(prot, 0, 1).numpy()
        cloud = pv.PolyData(prot)
        print(cloud)
        cloud.plot()
    

transforms = [UnitSphereNormalization(),
              Translate(p=0.8, scale=1),
              RotateAroundZero(p=0.8),
              GaussianNoise(n=100, T=2500, p=0.8),
              RandomSampler(sample_size=5000)]


def generate_train_valid_set(df, tfms, val_pct, seed=42, **kwargs):
    '''
    Can also take some other arguments to be passed to the dataset initializer

            -> path (str): path to the parent directory containing the train files 
    '''

    train_size = int(len(df) * (1 - val_pct))    
    df_train = df.sample(train_size, random_state=seed)
    df_valid = df.drop(df_train.index)

    return ProteinDataset(df_train, tfms, root=root, **kwargs), ProteinDataset(df_valid, tfms, root=root, **kwargs)

'''
def generate_train_valid_set_boring(df, val_pct, seed=42):
    train_size = int(len(df) * val_pct)    
    df_train = df.sample(train_size, random_state=seed)
    df_valid = df.drop(df_train.index)

    return df_train, df_valid
'''

'\ndef generate_train_valid_set_boring(df, val_pct, seed=42):\n    train_size = int(len(df) * val_pct)    \n    df_train = df.sample(train_size, random_state=seed)\n    df_valid = df.drop(df_train.index)\n\n    return df_train, df_valid\n'

In [60]:
example_dataframe = create_dataframe(raw_train_dataframe_f1, [2, 30, 10])
example_set_train, example_set_valid = generate_train_valid_set(example_dataframe, transforms, 0.2)

Using all data for the class 2
Using all data for the class 30
Using all data for the class 10


100%|██████████| 7/7 [00:00<00:00, 56.23it/s]


In [ ]:
example_set_train.__getitem__(0, raw=True)

tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(2., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(2., device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')


In [24]:
item = example_set_train.__getitem__(0, raw=True)
item[0].shape

torch.Size([3, 18730])

In [25]:
print(f'{item[0].min() = } - {item[0].max() = }')

item[0].min() = tensor(-42.3550, device='cuda:0') - item[0].max() = tensor(41.5100, device='cuda:0')


In [26]:
sampler = UnitSphereNormalization()
prot, _ = example_set_train.__getitem__(0, raw=True)
sampler(prot)

tensor([[ 0.0122,  0.0124,  0.0113,  ..., -0.0026, -0.0016, -0.0019],
        [-0.0048, -0.0050, -0.0046,  ...,  0.0003, -0.0013, -0.0007],
        [-0.0073, -0.0074, -0.0067,  ...,  0.0023,  0.0029,  0.0026]],
       device='cuda:0')

### PointNet class

In [8]:
import torch.nn as nn 
import torch.nn.functional as F
from symmetria.encoders.pointnet_encoder import PointNetEncoder
from symmetria.decoders.prediction_head import PredictionHead

if debug_pointnet:
    bs, sz = 1, 2048
    encoder = PointNetEncoder(use_bn=False)
    mock_x = torch.randn(bs, 3, sz)
    output = encoder.forward(mock_x)
    print(f'{Text(output, 'output'):inspect}')

    decoder = PredictionHead(1024, 96)
    output_decoder = decoder.forward(output)
    print(f'{Text(output_decoder, 'output_decoder'):inspect}')

class PointNet(nn.Module):
    def __init__(self, output_size, max_points, use_bn=False): # make it prettier like in segmenter
        super().__init__()
        self.encoder = PointNetEncoder(use_bn)
        self.max_points = max_points

        self.input_size = self.get_input_size()        
        self.output_size = output_size

        self.decoder = PredictionHead(self.input_size, self.output_size, use_bn)

    def get_input_size(self):
        mock_x = torch.randn(1, 3, self.max_points)
        return self.encoder(mock_x).shape[-1]

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)

        return x #SoftMax already inside the CrossEntropyLoss

### Config Definition

In [36]:
from datetime import datetime
from torch import optim

time_now = datetime.now()
time_now = time_now.strftime('%d%m%Y_%H%M%S')
_folder = !pwd

env = {}
env['project_folder'] = _folder[0]
env['project'] = 'shrec-2025'
env['run'] = env['project'] + '_' + time_now
#env['output_dir'] = os.path.join(env['project_folder'], 'wandb', 'run_' + env['run'])
if 'output' not in os.listdir(env['project_folder']):
    os.mkdir('output')
env['output_dir'] = Path(env['project_folder']) / 'output'

env['max_points'] = 4000

#env['dataset'] = 'train_set-2_cls-1000_images.csv' 
env['dataset'] = 'train_set-all.csv' 
#env['dataset_path'] = '/mnt/dataset/shrec-2025-protein-classification/v2-20250331'
env['dataset_path'] = root

ds_path = Path('./datasets') / env['dataset']
print(f'Creating a test dataset from {ds_path}')

test = pd.read_csv(ds_path, index_col=0)
print(f'Test dataset has {len(test)} rows')

Creating a test dataset from datasets/train_set-all.csv
Test dataset has 9244 rows


### Model Definition

In [33]:
env['device'] = 'cuda:0'
env['val_pct'] = 0.2
env['augmentations_on'] = True
env['aug'] = [Translate(p=0.8),
              UnitSphereNormalization(),
              RotateAroundZero(p=0.8),
              GaussianNoise(n=10, T=(env['max_points']//2),  p=0.8),
              RandomSampler(sample_size=env['max_points'])]

env['bs'] = 4
env['epochs'] = 20
env['lr'] = 1e-3

env['model'] = PointNet(output_size=len(test['class_id'].unique()), max_points=env['max_points']).to(torch.device(env['device']))
env['loss_func'] = nn.CrossEntropyLoss()
env['optimizer'] = optim.Adam(env['model'].parameters(), lr=env['lr'])

In [29]:
env

{'project_folder': '/home/giorgio/venvs/SHREC',
 'project': 'shrec-2025',
 'run': 'shrec-2025_01042025_161836',
 'output_dir': PosixPath('/home/giorgio/venvs/SHREC/wandb/shrec-2025_01042025_161836'),
 'max_points': 4000,
 'dataset': 'train_set-all.csv',
 'dataset_path': PosixPath('/mnt/dataset/shrec-2025-protein-classification/v2-20250331'),
 'device': 'cuda:0',
 'val_pct': 0.2,
 'augmentations_on': True,
 'aug': [<__main__.Translate at 0x7f98ac532060>,
 'bs': 4,
 'epochs': 20,
 'lr': 0.001,
 'model': PointNet(
   (encoder): PointNetEncoder(
     (input_transform): TNet(
       (shared_mlps): Sequential(
         (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
         (1): ReLU()
         (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
         (3): ReLU()
         (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
         (5): ReLU()
       )
       (linear): Sequential(
         (0): Linear(in_features=1024, out_features=512, bias=True)
         (1): ReLU()
         (2): L

#### Training Loop

In [30]:
import wandb

def train(train_dl, valid_dl, env):
    net = env['model']
    loss_func = env['loss_func']
    optimizer = env['optimizer']

    wandb.init(project=env['project'], name=env['run'], dir=env['output_dir'], config=env)

    for epoch in range(env['epochs']):
        print('/' * 20 + f' Epoch: {epoch + 1} ' + '/' * 20)
        for step, batch in enumerate(tqdm(train_dl)):
            input_p, target = batch

            net.zero_grad()

            output = net(input_p)
            #print(f'{Text(output, 'output'):content}')
            #print(f'{Text(target, 'target'):content}')
            
            loss = loss_func(output, target)
            wandb.log({'epoch': epoch,
                    'loss': loss})
            #print(f'Training loss: {loss}')
            loss.backward()
            optimizer.step()

        print('/' * 20 + ' Validation ' + '/' * 20)
        with torch.no_grad():
            val_loss = []
            acc = []
            for step, batch in enumerate(tqdm(valid_dl)):
                input_p, target = batch
                
                output = net(input_p)

                loss = loss_func(output, target).cpu()
                val_loss.append(loss)

                matching = [torch.argmax(i) == torch.argmax(j) for i, j in zip(output, target)]
                accuracy = matching.count(True)/len(matching)
                acc.append(accuracy)
            wandb.log({'epoch': epoch,
                    'step': step,
                    'val_loss': np.mean(val_loss),
                    'accuracy': np.mean(acc)})

    wandb.finish()

In [18]:
test_train_ds, test_valid_ds = generate_train_valid_set(test, env['aug'], env['val_pct'])
test_train_dl, test_valid_dl = DataLoader(test_train_ds, batch_size=env['bs'], shuffle=True), DataLoader(test_valid_ds, batch_size=env['bs'], shuffle=True)

100%|██████████| 1849/1849 [00:23<00:00, 77.43it/s]


In [35]:
train(test_train_dl, test_valid_dl, env)

//////////////////// Epoch: 1 ////////////////////


100%|██████████| 1849/1849 [00:18<00:00, 100.50it/s]


//////////////////// Validation ////////////////////


  0%|          | 0/463 [00:00<?, ?it/s]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported